### Общее описание

Данная тетрадка используются для объединения различных `.csv` файлов в единый нормализованный датафрейм.\
Почему это нужно? Зачастую выгрузка поступает в виде разрозненных файлов (в этом примере это будут продажи за каждый месяц), которые нужно объединить в один для дальнейшего анализа.

### Импорт библиотек

In [1]:
# Импортируем основные библиотеки
import pandas as pd
import numpy as np

# Для работы с системным окружением
from pathlib import Path

## 1 Загрузка и обработка датасетов

## 1.1 Загрузка файлов

Определяем путь к текущей рабочей директории.\
Также предварительно указываем папку в переменной ``, в которой хранятся наши `.csv` файлы.

In [2]:
# Наименование папки с выгрузками
data_folder = 'Выгрузка_продажи'

# Путь к текущей рабочей директории
work_dir = Path.cwd()

# Создаем переменную с расположение файлов выгрузок
data_dir = work_dir / data_folder

In [3]:
# Список объектов Path с файлами выгрузок
csv_files = list(data_dir.glob("*.txt"))

In [4]:
# Распаковываем и выводим список файлов, чтобы убедить что все определилось
print(f"Найдено {len(csv_files)} файла:")
print(*[f.stem for f in csv_files], sep="\n")

Найдено 44 файла:
2022_10_выгрузка
2022_11_выгрузка
2022_12_выгрузка
2022_1_выгрузка
2022_2_выгрузка
2022_3_выгрузка
2022_4_выгрузка
2022_5_выгрузка
2022_6_выгрузка
2022_7_выгрузка
2022_8_выгрузка
2022_9_выгрузка
2023_10_выгрузка
2023_11_выгрузка
2023_12_выгрузка
2023_1_выгрузка
2023_2_выгрузка
2023_3_выгрузка
2023_4_выгрузка
2023_5_выгрузка
2023_6_выгрузка
2023_7_выгрузка
2023_8_выгрузка
2023_9_выгрузка
2024_10_выгрузка
2024_11_выгрузка
2024_12_выгрузка
2024_1_выгрузка
2024_2_выгрузка
2024_3_выгрузка
2024_4_выгрузка
2024_5_выгрузка
2024_6_выгрузка
2024_7_выгрузка
2024_8_выгрузка
2024_9_выгрузка
2025_1_выгрузка
2025_2_выгрузка
2025_3_выгрузка
2025_4_выгрузка
2025_5_выгрузка
2025_6_выгрузка
2025_7_выгрузка
2025_8_выгрузка


In [5]:
# Создаем пустой список, в котором будут храниться датафреймы
df_list = []

In [6]:
# Т.к. файлов много, то читаем файлы в цикле
for file in csv_files:
    try:
        df = pd.read_csv(
            file,
            engine='python',
            sep='\t',
            thousands='\xa0',
            decimal=','
        )
        # (!!!) Позже в этом месте будет функция по нормализации каждого датафрейма
        df_list.append(df) # Присоединяем результат к списку
    except Exception as e:
        print(f"Ошибка при чтении файла {file.name}: {e}.")

## 1.2 Нормализация отдельных датафреймов

В связи с особенностями выгрузок в **1С** данные приведены в искаженном виде: присутствуеют лишние строк заголовок (группировки полей), а также строки отдельных группировок создают пустую строку.\
Ниже действия по очистке таблиц от таких строк.

In [7]:
# Функция для создания маски фильтра по полю `Номенклатура`
def nomenclature_mask(df):
    
    # Задаем условие 1: поле 'Номенклатура' не пустое
    condition_1 = df['Номенклатура'].notna() & (df['Номенклатура'] != '')

    # Условие 2: 'Номенклатура' НЕ содержится в значении df['Номенклатура']
    condition_2 = ~df['Номенклатура'].astype(str).str.contains('Номенклатура', na=False)

    # Возвращаем готовую маску
    return condition_1 & condition_2

In [8]:
# Задаем поля с категориями типов в пределах которых и будет осуществляться сдвиг
category_columns = ['Тип_1', 'Тип_2', 'Тип_3', 'Тип_4', 'Тип_5', 'Тип_6', 'Тип_7', 'Тип_8']

# Определяем числовые поля
cols_purpose = ['Количество', 'Сумма продажи', 'Прибыль']

In [9]:
# Функция для сдвига названий групп товаров влево
def left_shift_groups(df):
    
    # Получаем подматрицу с нужными колонками
    data = df[category_columns].to_numpy()

    # Создаём маску валидных значений (не NaN и не пустая строка)
    valid_mask = (~pd.isna(data)) & (data != '')

    # Создаём пустой макет массива для заполнения результатом.
    result = np.full(data.shape, None, dtype=object)

    # Построчный сдвиг
    for i in range(data.shape[0]):
        # Создаем список, содержащий только валидные значения
        valid_values = data[i, valid_mask[i]]
        # Вставляем их в макет (вставляется слева направо)
        result[i, :len(valid_values)] = valid_values

    # Обновляем датафрейм значениями полученной матрицы
    df.loc[:, category_columns] = result
    
    # Возвращаем обновленный датафрейм
    return df

In [10]:
# Функция для очистки строк по признаку `Номенклатура`
def clear_rows_by_nomenclature(df, i):
    '''
    Предварительно сохраняет ключевые суммы для последующего сравнения
    со значениям после обработки датафрейма (чтобы убедиться, что данные не изменились)
    '''
    # В выгрузке 1С итоговые суммы в последней строке
    df_sum_before = df.iloc[-1][cols_purpose]
    
    '''
    В этой части датафрейм фильтруется по маске, получемой в отдельной функции: удаляеется N-ая
    (чаще вторая) строка с 1С, которая отвечает за группировку в "шапке", которая приводит к
    появлению лишней строки. Также удаляются строки с пустой номенклатурой, - это строки шапок
    "подгруппировок" (в этом примере - столбца периода).
    '''
    df = df[nomenclature_mask]

    '''
    Далее необходимо "сдвинуть" содержимое колонок с наименования групп товаров, т.к. в 1С они изначально
    идут смещенными вправо, из-за чего нельзя выполнить корректную группировку по этому признаку.
    '''
    # Обновляем датафрейм значениями полученной матрицы
    df = left_shift_groups(df)

    '''
    Убедимся что ключевые суммы не изменились после всех изменений:
    '''
    df_sum_after = df[cols_purpose].sum()

    # Задаем допуски сравнения. В выгрузке могут присутствовать мусорные записи
    tolerance = {
        cols_purpose[0]: 1e-3,
        cols_purpose[1]: 1e-2,
        cols_purpose[2]: 1e-2
    }
    # Сраниваем суммы по заданным полям
    check_sums = all(
        np.isclose(
            df_sum_before[column], df_sum_after[column], rtol=0, atol=tolerance[column]
        ) for column in cols_purpose
    )
        
    if not check_sums:
        # Если суммы в каком-то датафрейме не сходятся
        print(f"❌ Не сходятся суммы в {csv_files[i].stem} ❌")
        print("До преобразований:")
        print(df_sum_before.to_dict())
        print("После:")
        print(df_sum_after.to_dict())
        print()
    else:
        print(f"✅ {csv_files[i].stem}")
        print("До преобразований:")
        print(df_sum_before.to_dict())
        print("После:")
        print(df_sum_after.to_dict())
        print()
        # Возвращаем результат
        return df

In [ ]:
# с нумерацией с 1
df_list = [clear_rows_by_nomenclature(df, i) for i, df in enumerate(df_list, start=0)]

## 1.3 Объединение датафреймов

Объединяем полученные списком датафреймы:

In [12]:
# Склеиваем датафреймы
df = pd.concat(
    df_list,
    ignore_index=True
)

## 1.4 Работа с данными (при необходимости)

### 1.4.1 Приведение типов данных

Для корректного работы с датой, а также во избежание ошибок при сохранении этого поля, приведем столбец `` к типо `datetime`

In [13]:
# Приводим поле к типу datetime
df['Период, месяц'] = pd.to_datetime(
    df['Период, месяц'],
    format='%d.%m.%Y %H:%M:%S'
)

Другие поля также можно было бы привести к категориальным или строковым типам данных, но в рамках этих задач это не имеет смысла, т.к. не сказывается ни на быстродействии, ни на размере файлов.\
Наименования полей также остаются без изменения, чтобы оставались понятными при необходимости загрузки `csv` в Excel для операторов 1С.

### 1.4.2 Удаление лишних групп товаров

В текущем примере идет подготовка данных для прогнозирования продаж и прибыли.\
Для этих целей нам необходимо исключить некоторые группы товаров из учета.

In [14]:
# Выводим список уникальных значений в наименованиях основных групп товаров
df['Тип_1'].unique()

array(['ФРУКТЫ, ОВОЩИ', 'АЛКОГОЛЬ', 'УЦЕНКА/СПИСАНИЕ НЕКОНДИЦИИ',
       'ЛИШНЕЕ', 'ПИВО И СЛ/А НАПИТ', 'ТАБАЧНЫЕ ИЗДЕЛИЯ (4СЕКЦИЯ)',
       'ЗАМОРОЖЕННЫЕ ПРОДУКТЫ', 'КОНСЕРВАЦИЯ', 'БЫТОВАЯ ХИМИЯ',
       'СРЕДСТВА ГИГИЕНЫ', 'КОСМЕТИКА, ПАРФЮМЕРИЯ', 'КАССА', 'NON FOOD',
       'ДЕТСКОЕ ПИТАНИЕ', 'МАЙОНЕЗЫ,КЕТЧУПЫ,СОУСЫ', 'БАКАЛЕЯ',
       'ТАЛНАХСКАЯ 22 ПОСТОРОННИЕ ПОСТАВЩИКИ',
       '24 СОБСТВЕННОЕ ПРОИЗВОДСТВО (СПК) (5 СЕКЦИЯ)',
       'ОХЛАЖДЕННЫЕ ПРОДУКТЫ', 'МОЛОЧНЫЕ ПРОДУКТЫ',
       'СНЭКИ,СУХОФРУКТЫ,ОРЕХИ', 'РЫБНЫЙ ГАСТРОНОМ И ПРЕСЕРВЫ', 'ПЕКАРНЯ',
       'МЯСНОЙ ГАСТРОНОМ', 'ДИЕТИЧЕСКИЕ ПРОДУКТЫ', 'КОНДИТЕРСКИЕ ИЗДЕЛИЯ',
       'ХОЗТОВАРЫ', 'ХОЗ.НУЖДЫ Д/МАГАЗИНОВ', 'НАПИТКИ Б/А', 'СЫР',
       'ЧАЙ,КОФЕ,КАКАО', 'ДИСКОНТНЫЕ КАРТЫ.', 'КОРМА ДЛЯ ЖИВОТНЫХ',
       'ГРИЛЬ', 'РАСТИТЕЛЬНЫЕ МАСЛА', 'ЯЙЦО', 'СЕЗОННЫЕ ИЗДЕЛИЯ',
       'ПРОДОВОЛЬСТВЕННЫЕ ТОВАРЫ', None], dtype=object)

В этом списке у нас исключаются такие группы как:
- `УЦЕНКА/СПИСАНИЕ НЕКОНДИЦИИ`;
- `ЛИШНЕЕ`;
- `ДИСКОНТНЫЕ КАРТЫ.`;
- `ХОЗ.НУЖДЫ Д/МАГАЗИНОВ`;
- `ТАЛНАХСКАЯ 22 ПОСТОРОННИЕ ПОСТАВЩИКИ`;
- `УСЛУГИ`;
- `СЕЗОННЫЕ ИЗДЕЛИЯ`.

In [15]:
# Задаем список наименований групп для удаления
trash_groups = [
    'УЦЕНКА/СПИСАНИЕ НЕКОНДИЦИИ', 'ЛИШНЕЕ', 'ДИСКОНТНЫЕ КАРТЫ.',
    'ХОЗ.НУЖДЫ Д/МАГАЗИНОВ', 'ТАЛНАХСКАЯ 22 ПОСТОРОННИЕ ПОСТАВЩИКИ',
    'УСЛУГИ', 'СЕЗОННЫЕ ИЗДЕЛИЯ'
]

# Фильтруем датафрейм через маску
df = df[
    ~df['Тип_1'].isin(trash_groups)
]

Убедимся что лишние группы были удалены:

In [16]:
# Количество строк, содержащих удаляемые наименования в группе
df['Тип_1'].isin(trash_groups).sum()

np.int64(0)

### 1.4.3 Агрегация по родительской группе

Прогноз выполняется для каждой из основных групп товаров поля `Тип_1`, поэтому при группировке по этому столбцу суммируем значения следующих полей:
- `'Количество'`;
- `'Сумма продажи'`;
- `'Прибыль'`.

In [17]:
# Задаем поля, по которым будет выполнятьяс группировка
groupby_columns = ['Период, месяц', 'Тип_1']

# Группируем исходный датафрейм
df = df.groupby(
    groupby_columns,
    as_index=False
)[cols_purpose].sum()

## 1.5 Сохранение результа

In [18]:
# Имя файла готового результата
output_file_name = "Продажи.csv"

# Расположение файла
file_save_path = work_dir / output_file_name

In [19]:
# Сохраняем в `csv` файл
df.to_csv(
    file_save_path,
    sep='\t',
    index=False
)